In [14]:
import json
import pandas as pd
from datetime import datetime, timezone

# obtains timestamp
def get_timestamp(filename):
  try:
    # takes the part before the dot
    ts_str = filename.split('.')[0]
    # converts to float
    ts_float = float(ts_str)
    # converts to UTC datetime
    return datetime.fromtimestamp(ts_float, tz=timezone.utc)
  except Exception:
    return None

# read JSON file
with open('data/segments_data_round3_day1_classification.json', 'r') as file:
  data = json.load(file)

# stores data for later conversion into dataframe
segments = []
drives = []
cameras = []
images = []
camera_images = []
categories = []
image_categories = []

# id maps
segment_id_map = {}
drive_id_map = {}
camera_id_map = {}
image_id_map = {}
category_id_map = {}

# counters for the id (beneficial for conversion to psql)
segment_counter = 1
drive_counter = 1
camera_counter = 1
image_counter = 1
cam_img_counter = 1
category_counter = 1
img_cat_counter = 1

# keeps track of missing classifications
missing_classifications = []

In [15]:
for segment_name, segment_data in data.items():
  # assign unique ID to each segment
  if segment_name not in segment_id_map:
    segment_id_map[segment_name] = segment_counter
    segment_counter += 1
  seg_pk = segment_id_map[segment_name]

  # track timestamps for the segments
  segment_timestamps = []

  for drive_name, drive_data in segment_data.items():
    # obtaining unique drive_id
    drive_key = (segment_name, drive_name)
    if drive_key not in drive_id_map:
      drive_id_map[drive_key] = drive_counter
      drive_counter += 1
    drive_pk = drive_id_map[drive_key]

    # saves dir_day and dir_pass
    dir_day = drive_data.get('dir_day')
    dir_pass = drive_data.get('dir_pass')
    # tracks timestampes
    drive_timestamps = []

    # add to Cameras table
    # finds all keys starting with 'cam'
    camera_keys = [k for k in drive_data.keys() if k.startswith('cam')]
    # obtains cam info and assign unique camera id
    for cam_name in camera_keys:
      cam_key = f"{drive_name}_{cam_name}_{segment_name}"
      if cam_key not in camera_id_map:
        camera_id_map[cam_key] = camera_counter
        camera_counter += 1
      cam_pk = camera_id_map[cam_key]

      cameras.append({
        "Camera_ID": cam_pk,
        "Drive_ID": drive_pk,
        "Name": cam_name
      })

      cam_data = drive_data[cam_name]

      # process images by color, depth
      for img_type in ['color', 'depth']:
        if img_type in cam_data:
          for filename in cam_data[img_type]:
            if filename not in image_id_map:
              image_id_map[filename] = image_counter
              ts = get_timestamp(filename)
              images.append({
                "Image_ID": image_counter,
                "Filename": filename,
                "Type": img_type,
                "Timestamp": ts
              })
              
              # track timestamps
              if ts:
                segment_timestamps.append(ts)
                drive_timestamps.append(ts)
              
              image_counter += 1
              
            image_pk = image_id_map[filename]
            
            camera_images.append({
              "ID": cam_img_counter,
              "Camera_ID": cam_pk,
              "Image_ID": image_pk
            })
            cam_img_counter += 1

      # process classifications
      if "Classification_Swin" in cam_data:
        for category_name, file_list in cam_data["Classification_Swin"].items():
          # obtains classifcation info and assigns unique id
          if category_name not in category_id_map:
            category_id_map[category_name] = category_counter
            categories.append({
              "Category_ID": category_counter,
              "Name": category_name
            })
            category_counter += 1
          cat_pk = category_id_map[category_name]
          
          for filename in file_list:
            # extracts filename
            filename_only = filename.split('\\')[-1]
            
            if filename_only in image_id_map:
              image_pk = image_id_map[filename_only]
              image_categories.append({
                "ID": img_cat_counter,
                "Image_ID": image_pk,
                "Category_ID": cat_pk,
                "Confidence": None
              })
              img_cat_counter += 1
            else:
              missing_classifications.append({
                "filename": filename_only,
                "category": category_name,
                "segment": segment_name,
                "drive": drive_name,
                "camera": cam_name
              })

    # adds to Drives table
    drives.append({
      "Drive_ID": drive_pk,
      "Name": drive_name,
      "Segment_ID": seg_pk,
      "Dir_Day": dir_day,
      "Dir_Pass": dir_pass,
      "Time_Driven": min(drive_timestamps) if drive_timestamps else None
    })

  # adds to Segments
  # using min of the timestamps
  seg_time_recorded = min(segment_timestamps) if segment_timestamps else None
  segments.append({
    "Segment_ID": seg_pk,
    "Name": segment_name,
    "Location": "Fort Wayne, IN",
    "Date_Recorded": seg_time_recorded.date() if seg_time_recorded else None
  })

In [16]:
# data validations
print("=" * 60)
print("DATA SUMMARY")
print("=" * 60)
print(f"Segments: {len(segments)}")
print(f"Drives: {len(drives)}")
print(f"Cameras: {len(cameras)}")
print(f"Unique Images: {len(images)}")
print(f"Camera-Image Relationships: {len(camera_images)}")
print(f"Categories: {len(categories)}")
print(f"Image-Category Relationships: {len(image_categories)}")

# check classification coverage
classified_image_ids = {ic['Image_ID'] for ic in image_categories}
all_image_ids = {i['Image_ID'] for i in images}
color_images = [i for i in images if i['Type'] == 'color']
print(f"\nImages with classifications: {len(classified_image_ids)}")
print(f"Total color images: {len(color_images)}")
print(f"Classification coverage: {len(classified_image_ids)/len(color_images)*100:.1f}%")

if missing_classifications:
  print(f"\nWARNING: {len(missing_classifications)} classified images not found in color/depth lists")
  print("First 5 missing:")
  for miss in missing_classifications[:5]:
    print(f"  - {miss['filename']} ({miss['category']}) in {miss['segment']}/{miss['drive']}/{miss['camera']}")
  
print("\n" + "=" * 60)

# convert to dataframes
segments_df = pd.DataFrame(segments)
drives_df = pd.DataFrame(drives)
cameras_df = pd.DataFrame(cameras)
images_df = pd.DataFrame(images)
camera_images_df = pd.DataFrame(camera_images)
categories_df = pd.DataFrame(categories)
image_categories_df = pd.DataFrame(image_categories)

# preview
print("\nSEGMENTS")
print(segments_df.head())

print("\nDRIVES (showing Segment_ID relationship)")
print(drives_df[['Drive_ID', 'Name', 'Segment_ID', 'Dir_Pass']].head(10))

print("\nCAMERAS")
print(cameras_df.head())

print("\nIMAGES")
print(images_df.head())

print("\nCAMERA_IMAGES")
print(camera_images_df.head())

print("\nCATEGORIES")
print(categories_df.head())

print("\nIMAGE_CATEGORIES")
print(image_categories_df.head())

DATA SUMMARY
Segments: 440
Drives: 504
Cameras: 1008
Unique Images: 400191
Camera-Image Relationships: 400197
Categories: 8
Image-Category Relationships: 299541

Images with classifications: 200061
Total color images: 200096
Classification coverage: 100.0%


SEGMENTS
   Segment_ID           Name        Location Date_Recorded
0           1   segment_7358  Fort Wayne, IN    2025-03-09
1           2   segment_7352  Fort Wayne, IN    2025-03-09
2           3   segment_7348  Fort Wayne, IN    2025-03-09
3           4   segment_7360  Fort Wayne, IN    2025-03-09
4           5  segment_26962  Fort Wayne, IN    2025-03-09

DRIVES (showing Segment_ID relationship)
   Drive_ID     Name  Segment_ID Dir_Pass
0         1  drive_0           1    pass0
1         2  drive_0           2    pass0
2         3  drive_0           3    pass0
3         4  drive_0           4    pass0
4         5  drive_0           5    pass0
5         6  drive_0           6    pass0
6         7  drive_0           7    pass0


CONNECTING DATAFRAMES TO PSQL (NEON TECH)

In [18]:
from sqlalchemy import create_engine
import os

conn_str = os.getenv("NEON_CONNECTION_STR")
engine = create_engine(conn_str)

segments_df.to_sql("segments", engine, if_exists="append", index=False)
drives_df.to_sql("drives", engine, if_exists="append", index=False)
cameras_df.to_sql("cameras", engine, if_exists="append", index=False)
images_df.to_sql("images", engine, if_exists="append", index=False)
camera_images_df.to_sql("camera_images", engine, if_exists="append", index=False)
categories_df.to_sql("categories", engine, if_exists="append", index=False)
image_categories_df.to_sql("image_categories", engine, if_exists="append", index=False)


541

In [19]:
check_segments = pd.read_sql("SELECT * FROM segments LIMIT 10;", engine)
check_segments

,Segment_ID,Name,Location,Date_Recorded
0,1,segment_7358,"Fort Wayne, IN",2025-03-09
1,2,segment_7352,"Fort Wayne, IN",2025-03-09
2,3,segment_7348,"Fort Wayne, IN",2025-03-09
3,4,segment_7360,"Fort Wayne, IN",2025-03-09
4,5,segment_26962,"Fort Wayne, IN",2025-03-09
5,6,segment_26934,"Fort Wayne, IN",2025-03-09
6,7,segment_26932,"Fort Wayne, IN",2025-03-09
7,8,segment_26974,"Fort Wayne, IN",2025-03-09
8,9,segment_26936,"Fort Wayne, IN",2025-03-09
9,10,segment_6834,"Fort Wayne, IN",2025-03-09
